# 03 - Optimisation durable
Nous testons plusieurs scénarios d'optimisation avec contraintes et traçons une frontière rendement contre carbone.

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd

from src.metrics import annualized_return
from src.optimization import optimize_portfolio

PROCESSED_DIR = pathlib.Path('..') / 'data' / 'processed'
returns = pd.read_csv(PROCESSED_DIR / 'returns.csv', index_col=0, parse_dates=True)
carbon_intensity = pd.read_csv(PROCESSED_DIR / 'carbon_intensity.csv', index_col=0).iloc[:, 0]

scenarios = []
carbon_grid = [80, 100, 120, 150, 200]
for max_carbon in carbon_grid:
    weights = optimize_portfolio(
        returns=returns,
        carbon_intensity=carbon_intensity,
        max_carbon=max_carbon,
    )
    port_returns = returns @ weights
    scenarios.append({
        'max_carbon': max_carbon,
        'annual_return': annualized_return(port_returns),
        'carbon_actual': float(carbon_intensity @ weights),
    })

frontier = pd.DataFrame(scenarios)
frontier.sort_values('carbon_actual', inplace=True)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(frontier['carbon_actual'], frontier['annual_return'], marker='o')
ax.set_xlabel('Intensité carbone effective')
ax.set_ylabel('Rendement annualisé')
ax.set_title('Frontière rendement vs carbone')
fig.tight_layout()
fig
